# VVP - Validation and Verification Protocol Tutorial

This tutorial demonstrates how to use the VVP (Validation and Verification Protocol) tool for input validation and logical structure verification.

## Overview

VVP provides:
- **Input validation** - Verify data format and structure
- **Rule validation** - Check logical rule structures
- **Pre-execution verification** - Ensure data integrity before processing
- **Format compliance** - Validate against expected schemas

VVP is typically used **before** other protocols (like REP) to ensure inputs are well-formed.

## Setup

In [ ]:
import subprocess
import json
from pathlib import Path

TOOL_PATH = Path("../code/tools/run_vvp_tool.py")

def validate_input(data):
    """Helper function to validate input data"""
    result = subprocess.run(
        ["python", str(TOOL_PATH), "--json", json.dumps(data)],
        capture_output=True,
        text=True
    )
    return json.loads(result.stdout)

## Example 1: Basic Rule Validation

Validating a simple rule structure.

In [ ]:
# Valid rule structure
data = {
    "rules": [
        [["premise1", "premise2"], "conclusion"]
    ]
}

result = validate_input(data)

print(f"Validation Status: {result['validation_status']}")
print(f"Reason: {result['reason']}")

## Example 2: Multiple Rules Validation

Validating multiple rule structures at once.

In [ ]:
# Multiple valid rules
data = {
    "rules": [
        [["All humans are mortal", "Socrates is human"], "Socrates is mortal"],
        [["All birds have feathers", "Penguins are birds"], "Penguins have feathers"],
        [["If it rains, the ground gets wet", "It is raining"], "The ground is wet"]
    ]
}

result = validate_input(data)

print(f"Validation Status: {result['validation_status']}")
print(f"Reason: {result['reason']}")

## Example 3: Detecting Invalid Rule Structures

VVP will catch malformed rules.

In [ ]:
# Invalid rule structure (missing conclusion)
invalid_data = {
    "rules": [
        [["premise1", "premise2"]]  # Missing conclusion
    ]
}

result = validate_input(invalid_data)

print(f"Validation Status: {result['validation_status']}")
print(f"Reason: {result['reason']}")

## Example 4: Empty Rules Validation

Handling edge cases like empty rule sets.

In [ ]:
# Empty rules
data = {
    "rules": []
}

result = validate_input(data)

print(f"Validation Status: {result['validation_status']}")
print(f"Reason: {result['reason']}")

## Example 5: Pre-Reasoning Validation Pipeline

Using VVP before REP to ensure valid inputs.

In [ ]:
# Reasoning data to validate
reasoning_data = {
    "reasoning_type": "deductive",
    "facts": [
        "All mammals are warm-blooded",
        "Whales are mammals"
    ],
    "rules": [
        [["All mammals are warm-blooded", "Whales are mammals"], "Whales are warm-blooded"]
    ]
}

# Step 1: Validate with VVP
print("Step 1: Validating input structure...")
validation_result = validate_input({"rules": reasoning_data["rules"]})
print(f"  Status: {validation_result['validation_status']}")

# Step 2: If valid, proceed to reasoning
if validation_result['validation_status'] == 'success':
    print("\nStep 2: Input validated. Ready for reasoning.")
    print("  (Would now pass to REP tool for actual reasoning)")
else:
    print(f"\nStep 2: Validation failed. Cannot proceed.")
    print(f"  Reason: {validation_result['reason']}")

## Example 6: Batch Validation

Validating multiple datasets in sequence.

In [ ]:
# Multiple datasets to validate
datasets = [
    {
        "name": "Dataset 1",
        "rules": [[["a", "b"], "c"]]
    },
    {
        "name": "Dataset 2",
        "rules": [[["x"], "y"], [["y", "z"], "w"]]
    },
    {
        "name": "Dataset 3 (invalid)",
        "rules": [[["invalid"]]]  # Missing conclusion
    }
]

print("Batch Validation Results:\n")
for dataset in datasets:
    result = validate_input({"rules": dataset["rules"]})
    status_icon = "✓" if result['validation_status'] == 'success' else "✗"
    print(f"{status_icon} {dataset['name']}: {result['validation_status']}")
    if result['validation_status'] != 'success':
        print(f"    Reason: {result['reason']}")

## Example 7: Command-Line Usage

VVP can be used directly from the command line.

In [ ]:
# Using command line with JSON input
!python ../code/tools/run_vvp_tool.py --json '{"rules": [[["premise1", "premise2"], "conclusion"]]}'

## Example 8: File Input Validation

Validating data from JSON files.

In [ ]:
# Create a validation input file
validation_input = {
    "rules": [
        [["All dogs are animals", "Rex is a dog"], "Rex is an animal"],
        [["All animals need food", "Rex is an animal"], "Rex needs food"]
    ]
}

with open('/tmp/validation_input.json', 'w') as f:
    json.dump(validation_input, f, indent=2)

# Validate from file
!python ../code/tools/run_vvp_tool.py --file /tmp/validation_input.json

## Example 9: Integration with Error Handling

Building robust pipelines with validation.

In [ ]:
def safe_reasoning_pipeline(reasoning_data):
    """
    Safely execute reasoning with pre-validation
    """
    print("Starting Safe Reasoning Pipeline...\n")
    
    # Extract rules for validation
    if 'rules' not in reasoning_data:
        print("❌ Error: No rules provided")
        return None
    
    # Step 1: Validate
    print("Step 1: Validating input structure...")
    validation_result = validate_input({"rules": reasoning_data["rules"]})
    
    if validation_result['validation_status'] != 'success':
        print(f"❌ Validation Failed: {validation_result['reason']}")
        return None
    
    print("✓ Validation passed\n")
    
    # Step 2: Execute reasoning (simulated)
    print("Step 2: Executing reasoning...")
    print("✓ Reasoning completed\n")
    
    return {"status": "success", "message": "Pipeline completed successfully"}

# Test with valid data
valid_data = {
    "reasoning_type": "deductive",
    "facts": ["fact1", "fact2"],
    "rules": [[["fact1", "fact2"], "conclusion"]]
}

result = safe_reasoning_pipeline(valid_data)
print(f"Final Result: {result}")

## Example 10: Complex Rule Structure Validation

Validating more complex logical structures.

In [ ]:
# Complex nested reasoning rules
complex_data = {
    "rules": [
        # Rule 1: Simple modus ponens
        [["If P then Q", "P"], "Q"],
        
        # Rule 2: Chained inference
        [["Q", "If Q then R"], "R"],
        
        # Rule 3: Multiple premises
        [["R", "S", "If R and S then T"], "T"]
    ]
}

result = validate_input(complex_data)

print(f"Complex Structure Validation:")
print(f"  Status: {result['validation_status']}")
print(f"  Reason: {result['reason']}")
print(f"  Rules Validated: {len(complex_data['rules'])}")

## Example 11: Stdin Pipe Usage

Using VVP in Unix pipelines.

In [ ]:
# Using stdin pipe
!echo '{"rules": [[["a", "b"], "c"]]}' | python ../code/tools/run_vvp_tool.py

## Example 12: Full Validation + Reasoning Workflow

Complete example showing VVP → REP integration.

In [ ]:
import subprocess

# Reasoning data
reasoning_data = {
    "reasoning_type": "deductive",
    "facts": [
        "All planets orbit stars",
        "Earth is a planet"
    ],
    "rules": [
        [["All planets orbit stars", "Earth is a planet"], "Earth orbits a star"]
    ]
}

print("=" * 60)
print("COMPLETE VALIDATION + REASONING WORKFLOW")
print("=" * 60)

# Phase 1: Validate with VVP
print("\n[PHASE 1: VALIDATION]")
validation_result = validate_input({"rules": reasoning_data["rules"]})
print(f"Status: {validation_result['validation_status']}")
print(f"Details: {validation_result['reason']}")

# Phase 2: Execute reasoning if validation passed
if validation_result['validation_status'] == 'success':
    print("\n[PHASE 2: REASONING]")
    print("Validation passed. Executing REP...")
    
    rep_result = subprocess.run(
        ["python", "../code/tools/run_rep_tool.py", "--json", json.dumps(reasoning_data)],
        capture_output=True,
        text=True
    )
    
    if rep_result.returncode == 0:
        rep_output = json.loads(rep_result.stdout)
        print(f"Reasoning Type: {rep_output['reasoning_type']}")
        print(f"Conclusions: {rep_output['conclusions']}")
        print(f"\n✓ Workflow completed successfully")
    else:
        print(f"❌ Reasoning failed: {rep_result.stderr}")
else:
    print("\n[PHASE 2: SKIPPED]")
    print("❌ Validation failed. Cannot proceed to reasoning.")

print("\n" + "=" * 60)

## Conclusion

The VVP tool provides essential validation capabilities for cognitive workflows. Key takeaways:

- **Pre-execution validation**: Always validate before processing
- **Error prevention**: Catch malformed inputs early
- **Pipeline integration**: Use as first step in reasoning workflows
- **Format compliance**: Ensure data meets expected schemas
- **Composability**: Easily chain with other protocols

## Best Practices

1. **Always validate before reasoning**: Use VVP before REP
2. **Handle validation failures gracefully**: Check status before proceeding
3. **Batch validate**: Validate multiple inputs efficiently
4. **Log validation results**: Track validation failures for debugging
5. **Use in CI/CD**: Validate data in automated pipelines

## Next Steps

- Combine VVP with REP for validated reasoning
- Use VVP with Five Laws Validator for complete governance
- Explore ESL for emotional validation
- Read the full documentation at [PAPER2AGENT_INTEGRATION.md](../PAPER2AGENT_INTEGRATION.md)